In [1]:
from sclblpy import main as sp

# Sclblpy 
Sclblpy allows you to upload a `.onnx` model to your sclbl admin that can subsequently be assigned to a device that has previously been registered on your account.


## (preliminary) make an account

To make an account, visit [Scailable](https://admin.sclbl.net/signup.html)

## (preliminary) register at least one device

To register a device, follow the tutorial [here](https://github.com/scailable/sclbl-tutorials/tree/master/solutions-manuals/sclbl-local-ai-manager). You'll be required to get a license as well - just apply for one and you'll receive a local evaluation copy (or use your previously obtained one, or get in contact with us)

If you've read the above already and are only looking for the command, it's `wget -q -O - https://get.sclbl.net | sudo bash` or `sudo /opt/sclbl/etc/init startui` if you've already installed the manager. NOTE: obviously do not run `sudo` commands if you're not sure what you're doing! 

NOTE: the device's device manager has to be running during this tutorial, otherwise the assignment will (somehwat obviously) not work.

## sign in

To use the package, you'll have to sign in with the username and password you've received in the previous step



## (optional) build a model

While most `.onnx` files will work out of the box with sclblpy, many of you will aim to use sclblpy to deploy some kind of Neural Network trained in e.g. Pytorch. We'll show you briefly how to export a Neural Network from the torch.nn.Module ... module to `onnx`, but feel free to use your own model, as long as it can be exported to `onnx` (one caveat: Not every operator is currently supported, find a list of the ones that work [here](https://github.com/scailable/sclblonnx/blob/master/sclblonnx/supported_onnx.json).

Our example won't even be trained, since we're only interested in the deployment part, essentially we'll be doing the first few steps from https://pytorch.org/tutorials/advanced/super_resolution_with_onnxruntime.html; feel free to do the rest of the tutorial as well!


NOTE: `pixel_shuffle()` is not yet supported by our runtime, so we have to remove it before conversion

In [2]:
import torch
import torch.nn as nn
import torch.nn.init as init


class SuperResolutionNet(nn.Module):
    def __init__(self, upscale_factor, inplace=False):
        super(SuperResolutionNet, self).__init__()

        self.relu = nn.ReLU(inplace=inplace)
        self.conv1 = nn.Conv2d(1, 64, (5, 5), (1, 1), (2, 2))
        self.conv2 = nn.Conv2d(64, 64, (3, 3), (1, 1), (1, 1))
        self.conv3 = nn.Conv2d(64, 32, (3, 3), (1, 1), (1, 1))
        self.conv4 = nn.Conv2d(32, upscale_factor ** 2, (3, 3), (1, 1), (1, 1))
        #self.pixel_shuffle = nn.PixelShuffle(upscale_factor)

        self._initialize_weights()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        #x = self.pixel_shuffle(self.conv4(x))
        return x

    def _initialize_weights(self):
        init.orthogonal_(self.conv1.weight, init.calculate_gain('relu'))
        init.orthogonal_(self.conv2.weight, init.calculate_gain('relu'))
        init.orthogonal_(self.conv3.weight, init.calculate_gain('relu'))
        init.orthogonal_(self.conv4.weight)

# Create the super-resolution model by using the above model definition.
torch_model = SuperResolutionNet(upscale_factor=3)

print(torch_model)

SuperResolutionNet(
  (relu): ReLU()
  (conv1): Conv2d(1, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(32, 9, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)


So now we have our model, we'll call torch.eval() before we convert it to onnx

In [3]:

batch_size = 1    # just a random number


# set the model to inference mode
torch_model.eval()

SuperResolutionNet(
  (relu): ReLU()
  (conv1): Conv2d(1, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(32, 9, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)

In [4]:
# Input to the model
x = torch.randn(batch_size, 1, 224, 224, requires_grad=True)
torch_out = torch_model(x)

# Export the model
torch.onnx.export(torch_model,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "super_resolution.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})

/home/thomas/anaconda3/lib/python3.7/site-packages/torch/onnx/utils.py:718: UserWarning: Provided key input for dynamic axes is not a valid input/output name
  warnings.warn("Provided key {} for dynamic axes is not a valid input/output name".format(key))
/home/thomas/anaconda3/lib/python3.7/site-packages/torch/onnx/utils.py:718: UserWarning: Provided key output for dynamic axes is not a valid input/output name
  warnings.warn("Provided key {} for dynamic axes is not a valid input/output name".format(key))


## write some docs

In [5]:
docs = {}
docs['name'] = "Faux Superresolution ONNX"
docs['documentation'] = "A long .md thing...."

## Upload the model to your admin (where we call the thing we upload stuff to?)

In [6]:
check = sp.upload_onnx("super_resolution.onnx","",docs)



#note: this will take a bit

Your ONNX file was successfully uploaded to Scailable!
NOTE: After transpiling, we will send you an email and your model will be available at https://admin.sclbl.net.
Or, alternatively, you can use the 'models()' function to list all your uploaded models. 



## (optional) wait for a few seconds to give the server time to convert the model

In [ ]:
import time

time.sleep(15)


#remove file
import os
if os.path.exists("super_resolution.onnx"):
    os.remove("super_resolution.onnx")
else:
    print("The file does not exist")

## (optional) check if the model has been uploaded

Go to https://admin.sclbl.net/index.html to check if your model has been uploaded.

Here you'll be able to check out your model's stats and assign a driver.

## get the id(s) of your device(s)

In order to interact with your devices, you'll need the id of the model you'll want to interact with. Luckily, that's easy as a pie. The `_verbose = False` suppresses written output, the `_return=True` forces the method to return the data


In [ ]:
devices = sp.devices(_return=True,_verbose=False)



`devices` will be a list of dictionaries containing a bunch of information about your devices, such as the id, did, rid, name, time of creation, runtime, ... However, this tutorial only requires `did` and `rid`

In [ ]:
device= [x for x in devices if x['name']=='My computer'][0]
device_did = device['did']
device_rid = device['rid']

## get the model's ID

In [ ]:
models = sp.models(_return=True, _verbose=False)
models

In [ ]:
model = [x for x in models if x['name'] == docs['name']][0]


In [ ]:
model_cfid = model['cfid']


In [ ]:
model

## Assign the model to a device



In [ ]:
sp.assign(model_cfid, device_did, device_rid, _verbose=True)


##  a model

In [ ]:

# Export the model
torch.onnx.export(torch_model,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "super_resolution_new.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input_new'],   # the model's input names
                  output_names = ['output_new'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})

In [ ]:
sp.update_onnx("super_resolution_new.onnx",model_cfid,"",docs)

In [ ]:

time.sleep(15)


#remove file
import os
if os.path.exists("super_resolution_new.onnx"):
    os.remove("super_resolution_new.onnx")
else:
    print("The file does not exist")

In [ ]:
models = sp.models(_return=True, _verbose=False)
model = [x for x in models if x['name'] == docs['name']][0]


In [ ]:
model

## models a model's docs

In [ ]:
docs = {}
docs['name'] = "a newly named Superresolution ONNX"
docs['documentation'] = "An even longer .md thing...."

sp.update_docs(model_cfid, docs)


## redeploy


## delte model

In [ ]:
sp.delete_model(model_cfid)  # Where cfid is the compute function id


In [ ]:
sp.models()